In [1]:
import time
import numpy as np
import pandas as pd
import json
import seaborn as sb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import seaborn as sb

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
import pickle

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import cluster_funcs as cf
import re

,state,qual_encoded,num_subjects,hours_tutoring,edu_length,rating_count,review_count,rating_av,bio_length,sunday_hours,...,sub_0,sub_1,sub_2,sub_3,sub_4,ed_0,ed_1,log_hourly_rate,exptext,exp_tot
0,CA,5,5,839.0,7,269,15.0,4.98885,376,0.583333,...,-0.033960,0.222921,-0.062145,-0.069276,0.001775,-0.016402,-0.031496,4.094345,1,16
1,CA,5,31,962.0,7,338,18.0,4.93195,323,0.500000,...,-0.070633,-0.036473,0.230497,-0.028919,0.076335,0.669648,0.020776,4.317488,13,22
2,CA,0,72,3899.0,4,1046,74.0,4.94349,248,1.000000,...,-0.043025,-0.138776,0.198153,-0.048833,0.140307,-0.017695,-0.031896,4.553877,0,30
3,CA,5,19,5592.0,7,1602,32.0,4.97940,417,0.000000,...,-0.223627,-0.112409,-0.049184,-0.233266,-0.042781,-0.022501,-0.033860,4.553877,7,7
4,CA,0,5,230.0,4,67,12.0,4.98507,173,0.041667,...,-0.190390,0.302855,-0.149778,0.040967,0.016708,-0.017356,-0.031909,4.828314,10,22


In [6]:
tutor_df.iloc[1]

qual_encoded         1.000000
background_check     0.000000
num_subjects        25.000000
hours_tutoring       3.000000
edu_length           7.000000
mean_hours           0.416667
state_cb             3.737146
qual_encoded_cb      3.913101
Name: 2049, dtype: float64

In [17]:
loaded_model = pickle.load(open('model_base.sav', 'rb'))
loaded_encodings = pickle.load(open('cb_encoding.sav', 'rb'))
np.exp(loaded_model.predict(tutor_df.iloc[3])[0]).round()

35.0

In [ ]:
"Do you hold any degrees? Choose from the list."
"Do you have an approved federal background check?"
"Approximately how many hours have you tutored in the past year? Enter a number."
"Which subjects do you intend to teach? Choose as many as you like from the list."
"Where do you plan to teach? Pick a state."
"How many hours each week are you available to teach?"

In [14]:
degree_list = ['Bachelors','Masters','MD','MBA','PhD','J.D.','Other']

def qual_encode(qual):
    
    qual_list = ['Masters','MBA','PhD','J.D.']
    qual_encodings = {'Masters': 1, 'MBA': 1, 'PhD': 2, 'J.D.': 3}
    if qual in qual_list:
        return qual_encodings[qual]
    else:
        return 0

In [71]:
qual1 = 'Bachelors'
qual2 = 'Masters'
qual3 = 'PhD'
all_quals = [qual1,qual2,qual3]
qual_tot = 0

for qual in all_quals:
    qual_tot += qual_encode(qual)
    
back_check = 1
hours_tutored = 200
subject_list = ['1','2','4','4','5','6','7','8','5','2']
num_subjects = len(subject_list)
edu_length = 3*len(all_quals) + 1
state = 'CA'
hours = 10
mean_hours = hours/(24*7)

In [72]:
cats = pd.Series({'state':state,'qual_encoded':qual_tot})
user_cats = loaded_encodings.transform(cats.to_frame().T).add_suffix('_cb')
user_input = pd.Series({'qual_encoded':qual_tot, 'background_check':back_check, 'num_subjects':num_subjects,
                          'hours_tutoring':hours_tutored, 'edu_length':edu_length, 'mean_hours': mean_hours})
u_input = user_input.to_frame().T.join(user_cats)
u_input

,qual_encoded,background_check,num_subjects,hours_tutoring,edu_length,mean_hours,state_cb,qual_encoded_cb
0,3.0,1.0,10.0,200.0,10.0,0.059524,3.871119,4.277966


In [73]:
np.exp(loaded_model.predict(u_input.values))[0].round()

52.0

In [6]:
def qual_encode(qual):
    
    qual_list = ['Masters','MBA','PhD','J.D.']
    qual_encodings = {'Masters': 5, 'MBA': 5, 'PhD': 20, 'J.D.': 30}
    if qual in qual_list:
        return qual_encodings[qual]
    else:
        return 0
    
def get_exp(bio):
    
    if 'years' in bio:
        yrs = np.array([int(i) for i in re.findall('\d+', bio)])
        if len(yrs[yrs<50])>0:
            return max(yrs[yrs<50])
        else:
            return 0
    else:
        return 0

In [4]:
tutor_df = pd.read_pickle("./tutors_outliers.pkl")
tutor_df.head()

,state,qual_encoded,num_subjects,hours_tutoring,edu_length,rating_count,review_count,rating_av,bio_length,sunday_hours,...,sub_0,sub_1,sub_2,sub_3,sub_4,ed_0,ed_1,log_hourly_rate,exptext,exp_tot
0,CA,5,5,839.0,7,269,15.0,4.98885,376,0.583333,...,-0.033960,0.222921,-0.062145,-0.069276,0.001775,-0.016402,-0.031496,4.094345,1,16
1,CA,5,31,962.0,7,338,18.0,4.93195,323,0.500000,...,-0.070633,-0.036473,0.230497,-0.028919,0.076335,0.669648,0.020776,4.317488,13,22
2,CA,0,72,3899.0,4,1046,74.0,4.94349,248,1.000000,...,-0.043025,-0.138776,0.198153,-0.048833,0.140307,-0.017695,-0.031896,4.553877,0,30
3,CA,5,19,5592.0,7,1602,32.0,4.97940,417,0.000000,...,-0.223627,-0.112409,-0.049184,-0.233266,-0.042781,-0.022501,-0.033860,4.553877,7,7
4,CA,0,5,230.0,4,67,12.0,4.98507,173,0.041667,...,-0.190390,0.302855,-0.149778,0.040967,0.016708,-0.017356,-0.031909,4.828314,10,22


In [27]:
loaded_model = pickle.load(open('model_xgb3.sav', 'rb'))
loaded_encodings = pickle.load(open('cbenc2.pkl', 'rb'))
tfidf_sub = pickle.load(open('tfidf.sav', 'rb'))
tfidf_ed = pickle.load(open('tfidf_ed.sav', 'rb'))
subPCA = pickle.load(open('subPCA.pkl', 'rb'))
edPCA = pickle.load(open('edPCA.pkl', 'rb'))

In [3]:
tutor_df.columns

NameError: name 'tutor_df' is not defined

In [57]:
tutor_df['bio_length'].median()

303.0

In [22]:
state = 'CA'

qual1 = 'Bachelors'
qual2 = 'Masters'
qual3 = 'PhD'
inst1 = 'University of Texas at Austin'
inst2 = 'Texas State University'
inst3 = 'Harvard'

subs = ['Algebra Calculus Prealgebra Geometry Trigonometry']
all_quals = [qual1,qual2,qual3]
qual_tot = 0

for qual in all_quals:
    qual_tot += qual_encode(qual)
    
years_exp = 10
rating_count = 10
rating_av = 4.895
review_count = 1
bio_length = 303
back_check = 1

sunday = 5/24
monday = 5/24
tuesday = 5/24
wednesday = 5/24
thursday = 5/24
friday = 5/24
saturday = 5/24

hours_tutored = 200
subject_list = ['1','2','4','4','5','6','7','8','5','2']
num_subjects = len(subject_list)
edu_length = 3*len(all_quals) + 1

In [8]:
def all_inst(inst1,inst2,inst3):

    inst = inst1 + ' ' + inst2 + ' ' + inst3 
    stopwords = ['university','University','of','in','College','college']
    for word in stopwords:
        inst = inst.replace(word,'').strip(' ')
    return inst

In [9]:
inst = 'Texas at Aust Stanford Harvard'

In [10]:
text_ed = tfidf_ed.transform([inst])

In [11]:
text_ed.shape

(1, 20000)

In [12]:
edPCA.transform(text_ed.todense())

array([[-0.03645394, -0.07087529]])

In [13]:
text_sub = tfidf_sub.transform(subs)
ed_df = pd.Series(edPCA.transform(text_ed.todense())[0]).to_frame().T
ed_df = ed_df.rename(columns={0:'ed_0',1:'ed_1'})

In [14]:
subPCA.transform(text_sub.todense())

array([[ 0.17430182,  0.11032639, -0.04469268, -0.01052112, -0.13713144]])

In [15]:
sub_df = pd.Series(subPCA.transform(text_sub.todense())[0]).to_frame().T
sub_df = sub_df.rename(columns={0:'sub_0',1:'sub_1',2:'sub_2',3:'sub_3',4:'sub_4'})

In [16]:

def is_ivy(inst1,inst2,inst3):
    ivys = ['Brown', 'Columbia', 'Dartmouth', 'Harvard', 'Cornell', 'Pennsylvania', 'Princeton', 'Yale']
    ivy = []
    for inst in [inst1,inst2,inst3]:
        if any(x in inst for x in ivys):
            ivy.append(1)
        else:
            ivy.append(0)
    return ivy

In [17]:
ivy = is_ivy(inst1,inst2,inst3)

In [18]:
cats = pd.Series({'state':state,'ivy_1e':ivy[0],'ivy_2e':ivy[1],'ivy_3e':ivy[2]})
user_cats = loaded_encodings.transform(cats.to_frame().T).add_suffix('_cb')

In [19]:
user_cats

,state_cb,ivy_1e_cb,ivy_2e_cb,ivy_3e_cb
0,3.912955,3.816676,3.818395,4.11391


In [20]:
user_cats[['ivy_1e_cb', 'state_cb', 'ivy_2e_cb', 'ivy_3e_cb']]

,ivy_1e_cb,state_cb,ivy_2e_cb,ivy_3e_cb
0,3.816676,3.912955,3.818395,4.11391


In [23]:
user_input = pd.Series({'qual_encoded':qual_tot, 'background_check':back_check, 'num_subjects':num_subjects,
                          'hours_tutoring':hours_tutored, 'edu_length':edu_length, 'bio_length': bio_length,
                        'sunday_hours': sunday,'monday_hours': monday, 'tuesday_hours': tuesday,
                        'wednesday_hours': wednesday,'thursday_hours': thursday, 'friday_hours': friday, 
                        'saturday_hours': saturday, 'rating_count':rating_count, 'rating_av': rating_av,
                       'review_count':review_count,'experience':years_exp, 'exptext': years_exp, 
                        'exp_tot':years_exp})
u_input = user_input.to_frame().T.join(user_cats).join(sub_df).join(ed_df)

In [24]:
u_input = u_input[['qual_encoded', 'num_subjects', 'hours_tutoring', 'edu_length',
       'rating_count', 'review_count', 'rating_av', 'bio_length',
       'sunday_hours', 'monday_hours', 'tuesday_hours', 'wednesday_hours',
       'thursday_hours', 'friday_hours', 'saturday_hours', 'experience',
       'sub_0', 'sub_1', 'sub_2', 'sub_3', 'sub_4', 'ed_0', 'ed_1', 'exptext',
       'exp_tot', 'state_cb', 'ivy_1e_cb', 'ivy_2e_cb', 'ivy_3e_cb']]
u_input

,qual_encoded,num_subjects,hours_tutoring,edu_length,rating_count,review_count,rating_av,bio_length,sunday_hours,monday_hours,...,sub_3,sub_4,ed_0,ed_1,exptext,exp_tot,state_cb,ivy_1e_cb,ivy_2e_cb,ivy_3e_cb
0,25.0,10.0,200.0,10.0,10.0,1.0,4.895,303.0,0.208333,0.208333,...,-0.010521,-0.137131,-0.036454,-0.070875,10.0,10.0,3.912955,3.816676,3.818395,4.11391


In [28]:
np.exp(loaded_model.predict(u_input.values))[0].round()

65.0

In [31]:
import utils.proc_funcs

ModuleNotFoundError: No module named 'utils'